In [ ]:
'''
# Fortonoume ta dataset
file_path_1 = r'C:\Users\Βαγγέλης\Desktop\Σχολή\4o Έτος\7ο Εξάμηνο\Ειδικό Θέμα με Τουσίδου (ομαδικό με Σταμάτη)\Datasets\Dataset_6\cleaned_books_dataset.csv'
books_data = pd.read_csv(file_path_1)

file_path_2 = r'C:\Users\Βαγγέλης\Desktop\Σχολή\4o Έτος\7ο Εξάμηνο\Ειδικό Θέμα με Τουσίδου (ομαδικό με Σταμάτη)\pythonProject1\cleaned_books.csv'
new_books_data = pd.read_csv(file_path_2)

file_path_3 = r'C:\Users\Βαγγέλης\Desktop\Σχολή\4o Έτος\7ο Εξάμηνο\Ειδικό Θέμα με Τουσίδου (ομαδικό με Σταμάτη)\pythonProject1\cleaned_user_ratings.csv'
user_ratings_data = pd.read_csv(file_path_3)
'''

In [2]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
from fuzzywuzzy import process, fuzz
from sklearn.preprocessing import MinMaxScaler
from scipy.sparse import csr_matrix, vstack
from scipy.sparse.linalg import svds
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import accuracy_score,precision_score, recall_score, f1_score
from tensorflow import keras
from tensorflow.keras import layers

/home/stmetaxas/.virtualenvs/Algorithm start/lib/python3.10/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
2025-01-24 00:23:25.492654: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-24 00:23:25.509209: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1737678205.529254  145079 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:

In [3]:
file_path_1 = '/home/stmetaxas/new_datasets/cleaned_books_dataset.csv'
books_data = pd.read_csv(file_path_1)

file_path_2 = '/home/stmetaxas/new_datasets/cleaned_books.csv'
new_books_data = pd.read_csv(file_path_2)

file_path_3 = '/home/stmetaxas/new_datasets/cleaned_user_ratings.csv'
user_ratings_data = pd.read_csv(file_path_3)

In [4]:
duplicate_isbns = books_data[books_data['isbn'].isin(new_books_data['ISBN'])]
unique_new_books_data = new_books_data[~new_books_data['ISBN'].isin(books_data['isbn'])]
unique_new_books_data = unique_new_books_data.rename(columns={
    'ISBN': 'isbn',
    'Title': 'title',
    'Author': 'authors',
    'Publication_Date': 'publication_date',
    'Publisher': 'publisher'
})

for column in books_data.columns:
    if column not in unique_new_books_data.columns:
        unique_new_books_data[column] = np.nan

unique_new_books_data = unique_new_books_data[books_data.columns]

In [5]:
books_data = pd.concat([books_data, unique_new_books_data], ignore_index=True)
books_data = books_data.drop_duplicates(subset='title', keep='first').reset_index(drop=True)
books_data['title'] = books_data['title'].fillna('unknown title').astype(str).str.lower().str.strip()
books_data['authors'] = books_data['authors'].fillna('unknown author').astype(str).str.lower().str.strip()
books_data = books_data[books_data['title'].str.len() >= 3]
books_data = books_data.dropna(subset=['title', 'authors']).drop_duplicates(subset='title').reset_index(drop=True)
books_data['title'] = books_data['title'].str.replace(r'\\', '', regex=True)
books_data.replace(to_replace='J.K. Rowling-Mary GrandPré', value='J.K. Rowling', inplace=True)

In [6]:
books_data['combined_features'] = (
        books_data['title'] + " " + books_data['title'] + " " + books_data['title'] +
        " " + books_data['authors'] + " " + books_data['authors']
).fillna('')

if 'weighted_rating' not in books_data.columns:
    books_data['weighted_rating'] = 0

scaler = MinMaxScaler()
books_data['weighted_rating'] = scaler.fit_transform(books_data[['weighted_rating']])
books_data['weighted_rating'] = books_data['weighted_rating'].fillna(0)

if 'rating_count' not in books_data.columns:
    books_data['rating_count'] = books_data.groupby('isbn')['isbn'].transform('count')
books_data['rating_count'] = books_data['rating_count'].fillna(0)

In [7]:
user_rating_counts = user_ratings_data['User_ID'].value_counts()
book_rating_counts = user_ratings_data['ISBN'].value_counts()

filtered_user_ratings = user_ratings_data[
    user_ratings_data['User_ID'].isin(user_rating_counts[user_rating_counts >= 4].index) &
    user_ratings_data['ISBN'].isin(book_rating_counts[book_rating_counts >= 4].index)
    ]
user_ratings_data = filtered_user_ratings

In [8]:
train_books_data, test_books_data = train_test_split(books_data, test_size=0.4, random_state=42)
train_ratings_data, test_ratings_data = train_test_split(user_ratings_data, test_size=0.4, random_state=42)

train_books_data = train_books_data.reset_index(drop=True)
test_books_data = test_books_data.reset_index(drop=True)
train_ratings_data = train_ratings_data.reset_index(drop=True)
test_ratings_data = test_ratings_data.reset_index(drop=True)

#Krataw mono ta ISBN pou xrhsimopoihthikan sto train set
train_users = train_ratings_data['User_ID'].unique()
train_isbns = train_ratings_data['ISBN'].unique()
test_ratings_data = test_ratings_data[
    test_ratings_data['User_ID'].isin(train_users) &
    test_ratings_data['ISBN'].isin(train_isbns)
    ]
test_ratings_data = test_ratings_data.reset_index(drop=True)

train_user_item_matrix = train_ratings_data.pivot(index='User_ID', columns='ISBN', values='Rating').fillna(0)
test_user_item_matrix = test_ratings_data.pivot(index='User_ID', columns='ISBN', values='Rating').fillna(0)

In [9]:
tfidf = TfidfVectorizer(stop_words='english', ngram_range=(1, 2), max_df=0.8, min_df=2)
tfidf_matrix_train = tfidf.fit_transform(train_books_data['combined_features'])

n_components = 250
svd = TruncatedSVD(n_components=n_components, random_state=42)
reduced_tfidf_matrix_train = svd.fit_transform(tfidf_matrix_train)

weighted_ratings_train = train_books_data['weighted_rating'].values.reshape(-1, 1)
train_books_data['popularity_penalty'] = 1/(1+train_books_data['rating_count'])
popularity_penalty_train = train_books_data['popularity_penalty'].values.reshape(-1,1)

final_feature_matrix_train = np.hstack((reduced_tfidf_matrix_train, weighted_ratings_train, popularity_penalty_train))
final_feature_matrix_train = csr_matrix(final_feature_matrix_train)

knn_content = NearestNeighbors(metric='cosine', algorithm='brute', n_jobs=-1)
knn_content.fit(final_feature_matrix_train)

NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1)

In [10]:
tfidf_matrix_test = tfidf.transform(test_books_data['combined_features'])
reduced_tfidf_matrix_test = svd.transform(tfidf_matrix_test)

weighted_ratings_test = test_books_data['weighted_rating'].values.reshape(-1,1)
test_books_data['popularity_penalty'] = 1/(1+test_books_data['rating_count'])
popularity_penalty_test = test_books_data['popularity_penalty'].values.reshape(-1,1)

final_feature_matrix_test = np.hstack((reduced_tfidf_matrix_test, weighted_ratings_test, popularity_penalty_test))
final_feature_matrix_test = csr_matrix(final_feature_matrix_test)

final_feature_matrix_combined = vstack([final_feature_matrix_train, final_feature_matrix_test])

combined_books_data = pd.concat([train_books_data, test_books_data], ignore_index=True)

knn_content_combined = NearestNeighbors(metric='cosine', algorithm='brute', n_jobs=-1)
knn_content_combined.fit(final_feature_matrix_combined)

NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1)

In [11]:
train_sparse_user_item = csr_matrix(train_user_item_matrix.values)

k = 15
U, sigma, Vt = svds(train_sparse_user_item, k=k)
sigma = np.diag(sigma)

predicted_ratings_matrix = np.dot(np.dot(U, sigma), Vt)
predicted_ratings_df = pd.DataFrame(predicted_ratings_matrix, index=train_user_item_matrix.index, columns=train_user_item_matrix.columns)

knn_cf = NearestNeighbors(metric='cosine', algorithm='brute')
knn_cf.fit(train_sparse_user_item)

NearestNeighbors(algorithm='brute', metric='cosine')

In [12]:
def train_simple_feedforward(features, embedding_dim=256, epochs=30, batch_size=256):
    if isinstance(features, csr_matrix):
        features = features.toarray()

    input_dim = features.shape[1]

    inputs = keras.Input(shape=(input_dim,))
    x = layers.Dense(embedding_dim, activation='relu')(inputs)
    outputs = layers.Dense(input_dim, activation='linear')(x)

    model = keras.Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer='adam', loss='mse')

    model.fit(
        features,
        features,
        epochs=epochs,
        batch_size=batch_size,
        validation_split=0.1,
        verbose=1
    )

    encoder = keras.Model(inputs=inputs, outputs=x)

    return model, encoder

In [13]:
model, encoder_model = train_simple_feedforward(
    final_feature_matrix_train,
    embedding_dim=256,
    epochs=30,
    batch_size=256
)

if isinstance(final_feature_matrix_train, csr_matrix):
    train_features_array = final_feature_matrix_train.toarray()
else:
    train_features_array = final_feature_matrix_train

book_embeddings_train = encoder_model.predict(train_features_array)

knn_nn_model_train = NearestNeighbors(metric='cosine')
knn_nn_model_train.fit(book_embeddings_train)

W0000 00:00:1737678265.789907  145079 gpu_device.cc:2344] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Epoch 1/30
509/509 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - loss: 5.2499e-04 - val_loss: 1.0259e-04
Epoch 2/30
509/509 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 7.5118e-05 - val_loss: 3.2494e-05
Epoch 3/30
509/509 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - loss: 2.7709e-05 - val_loss: 2.0808e-05
Epoch 4/30
509/509 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - loss: 1.9679e-05 - val_loss: 1.8480e-05
Epoch 5/30
509/509 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - loss: 1.7895e-05 - val_loss: 1.7437e-05
Epoch 6/30
509/509 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - loss: 1.6019e-05 - val_loss: 1.5088e-05
Epoch 7/30
509/509 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - loss: 1.4030e-05 - val_loss: 1.4503e-05
Epoch 8/30
509/509 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - loss: 1.3576e-05 - val_loss: 1.3735e-05
Epoch 9/30
509/509 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - loss: 1.3401e-05 - val_loss: 1.3222e-05
Epoch 10/30
509/509 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - loss: 1.2261e-05 - val_loss: 1.1471e-05
Epoch 11/30
509/509 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - loss

NearestNeighbors(metric='cosine')

In [14]:
model_combined, encoder_model_combined = train_simple_feedforward(
    final_feature_matrix_combined,
    embedding_dim=256,
    epochs=30,
    batch_size=256
)

if isinstance(final_feature_matrix_combined, csr_matrix):
    combined_array = final_feature_matrix_combined.toarray()
else:
    combined_array = final_feature_matrix_combined

book_embeddings_combined = encoder_model_combined.predict(combined_array)

knn_nn_model_combined = NearestNeighbors(metric='cosine', algorithm='brute')
knn_nn_model_combined.fit(book_embeddings_combined)

Epoch 1/30
847/847 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - loss: 4.0482e-04 - val_loss: 3.7157e-05
Epoch 2/30
847/847 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - loss: 2.2845e-05 - val_loss: 9.1447e-06
Epoch 3/30
847/847 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step - loss: 7.7750e-06 - val_loss: 6.9420e-06
Epoch 4/30
847/847 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step - loss: 6.2550e-06 - val_loss: 6.3925e-06
Epoch 5/30
847/847 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - loss: 5.5582e-06 - val_loss: 5.1437e-06
Epoch 6/30
847/847 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - loss: 4.7021e-06 - val_loss: 4.8683e-06
Epoch 7/30
847/847 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - loss: 4.5342e-06 - val_loss: 4.7091e-06
Epoch 8/30
847/847 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step - loss: 4.3515e-06 - val_loss: 4.5329e-06
Epoch 9/30
847/847 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - loss: 4.2415e-06 - val_loss: 4.4811e-06
Epoch 10/30
847/847 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - loss: 4.1128e-06 - val_loss: 4.3461e-06
Epoch 11/30
847/847 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - loss

NearestNeighbors(algorithm='brute', metric='cosine')

In [15]:
def get_user_input():
    user_input_title = input("Enter a title based on which I'll give you recommendations: ").strip().lower()
    return user_input_title

In [16]:
def fuzzy_match(query, choices, threshold=55, scorer=fuzz.partial_ratio):
    raw_matches = process.extract(query, choices, scorer=scorer)
    matches = [(choice[0], choice[1]) for choice in raw_matches if choice[1] >= threshold]
    return matches

In [17]:
def get_custom_recommendations(user_input_title, books_df, tfidf, svd, knn_model, top_n=20):
    choices = books_df['title'].tolist()
    matches = fuzzy_match(user_input_title, choices)
    if not matches:
        return []
    matched_title = matches[0][0]
    book_idx_list = books_df[books_df['title'] == matched_title].index
    if len(book_idx_list) == 0:
        return []
    book_idx = book_idx_list[0]

    distances, indices = knn_model.kneighbors(final_feature_matrix_train[book_idx], n_neighbors=top_n+1)
    results = []
    for i in range(1, len(indices.flatten())):
        title = books_df.iloc[indices.flatten()[i]]['title']
        score = 1 - distances.flatten()[i]
        results.append((title, score))
    return results

In [18]:
def get_nn_recommendations(
        user_input_title,
        books_df,
        encoder,
        knn_model,
        book_embeddings,
        feature_matrix,
        top_n=10
):
    query = user_input_title.strip().lower()
    possible_titles = books_df['title'].tolist()
    matches = process.extract(query, possible_titles, scorer=fuzz.partial_ratio, limit=1)
    if matches:
        print("Fuzzy match gave us:", matches)
    if not matches:
        print("No matching titles found.")
        return []
    matched_title = matches[0][0]
    print("Matched title is:", matched_title)
    matched_indices = books_df[books_df['title'] == matched_title].index
    if len(matched_indices) == 0:
        return []

    book_idx = matched_indices[0]

    if isinstance(feature_matrix, csr_matrix):
        book_vector = feature_matrix[book_idx].toarray()
    else:
        book_vector = feature_matrix[book_idx].reshape(1, -1)
    encoded_book = encoder.predict(book_vector)

    distances, indices = knn_model.kneighbors(encoded_book, n_neighbors=top_n+1)
    distances = distances.flatten()
    indices = indices.flatten()

    results = []
    for i in range(1, len(indices)):
        rec_idx = indices[i]
        rec_title = books_df.iloc[rec_idx]['title']
        similarity = 1.0 - distances[i]
        results.append((rec_title, similarity))

    return results

In [19]:
def get_collaborative_recommendations(user_ids, predicted_ratings_df, books_df, top_n=20):
    recommendations = pd.Series(dtype=float)
    for user_id in user_ids:
        if user_id not in predicted_ratings_df.index:
            continue
        user_ratings = predicted_ratings_df.loc[user_id].sort_values(ascending=False)
        known_ratings = train_user_item_matrix.loc[user_id]
        unseen_ratings = user_ratings[known_ratings == 0]
        recommendations = recommendations.add(unseen_ratings, fill_value=0)

    recommendations = recommendations.sort_values(ascending=False).head(top_n)
    rec_with_titles = []
    for isbn, score in recommendations.items():
        if isbn in books_df['isbn'].values:
            title = books_df.loc[books_df['isbn'] == isbn, 'title'].values[0]
            rec_with_titles.append((title, score))
    return rec_with_titles

In [20]:
def get_knn_recommendations(user_ids, knn_cf_model, books_df, top_n=20, num_neighbors=30):
    recommendations = pd.Series(dtype=float)
    train_matrix = csr_matrix(train_user_item_matrix.values)
    for user_id in user_ids:
        if user_id not in train_user_item_matrix.index:
            continue
        user_idx = train_user_item_matrix.index.get_loc(user_id)
        distances, indices = knn_cf_model.kneighbors(train_matrix[user_idx], n_neighbors=num_neighbors+1)
        similar_user_indices = indices.flatten()[1:]
        similar_user_ratings = train_user_item_matrix.iloc[similar_user_indices]
        avg_ratings = similar_user_ratings.mean(axis=0)
        unseen_ratings = avg_ratings[train_user_item_matrix.loc[user_id] == 0]
        recommendations = recommendations.add(unseen_ratings, fill_value=0)

    recommendations = recommendations.sort_values(ascending=False).head(top_n)
    rec_with_titles = []
    for isbn, score in recommendations.items():
        if isbn in books_df['isbn'].values:
            title = books_df.loc[books_df['isbn'] == isbn, 'title'].iloc[0]
            rec_with_titles.append((title, score))
    return rec_with_titles

In [21]:
def hybrid_recommendations(user_input_title, dataset='train', num_recommendations=20, user_ids=None):
    if dataset == 'train':
        current_books_df = train_books_data
        current_predicted_ratings_df = predicted_ratings_df
        current_knn_cf = knn_cf
        if user_ids is None:
            user_ids = np.random.choice(train_user_item_matrix.index, size=min(50,len(train_user_item_matrix.index)), replace=False)
        content_recs = get_custom_recommendations(user_input_title, current_books_df, tfidf, svd, knn_content, top_n=num_recommendations)
        nn_content_recs = get_nn_recommendations(user_input_title,train_books_data,encoder_model,knn_nn_model_train,book_embeddings_train,final_feature_matrix_train,top_n=num_recommendations)

    else:
        current_books_df = test_books_data
        current_predicted_ratings_df = predicted_ratings_df
        current_knn_cf = knn_cf
        if user_ids is None:
            if len(test_user_item_matrix.index) > 0:
                user_ids = np.random.choice(test_user_item_matrix.index, size=min(50, len(test_user_item_matrix.index)), replace=False)
            else:
                print("No users found in test set.")
                user_ids = []

        content_recs = get_custom_recommendations(user_input_title, combined_books_data, tfidf, svd, knn_content_combined, top_n=num_recommendations)
        nn_content_recs = get_nn_recommendations(user_input_title,combined_books_data,encoder_model,knn_nn_model_combined,book_embeddings_combined,final_feature_matrix_combined,top_n=num_recommendations)

    collaborative_recs = get_collaborative_recommendations(user_ids, current_predicted_ratings_df, current_books_df, top_n=num_recommendations)
    if not collaborative_recs: 
        print("Fallback Collaborative Recommendations to popular books")
        collaborative_recs = current_books_df[['title']].head(num_recommendations).apply(
            lambda x: (x['title'], 0), axis=1).tolist()

    knn_recs = get_knn_recommendations(user_ids, current_knn_cf, current_books_df, top_n=num_recommendations)
    if not knn_recs:
        print("Fallback KNN Recommendations to popular books")
        knn_recs = current_books_df[['title']].head(num_recommendations).apply(
            lambda x: (x['title'], 0), axis=1).tolist()

    print(f"\n--- {dataset.capitalize()} Content-Based Recommendations ---", content_recs)
    print(f"\n--- {dataset.capitalize()} NN-Based Content Recommendations ---", nn_content_recs)
    print(f"\n--- {dataset.capitalize()} Collaborative Recommendations ---", collaborative_recs)
    print(f"\n--- {dataset.capitalize()} KNN Recommendations ---", knn_recs)

    return {
        'content_recs': [title for title, _ in content_recs],
        'nn_content_recs': [title for title, _ in nn_content_recs],
        'collaborative_recs': [title for title, _ in collaborative_recs],
        'knn_recs': [title for title, _ in knn_recs]
    }

In [22]:
user_title = get_user_input()

print("\n--- Model Testing on Training Set ---")
train_results = hybrid_recommendations(user_input_title=user_title, dataset='train', num_recommendations=15)

print("\n--- Model Testing on Test Set ---")
test_results = hybrid_recommendations(user_input_title=user_title, dataset='test', num_recommendations=15)


--- Model Testing on Training Set ---
Fuzzy match gave us: [('harry potter y el misterio del príncipe (harry potter  #6)', 100)]
Matched title is: harry potter y el misterio del príncipe (harry potter  #6)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step

--- Train Content-Based Recommendations --- [('harry potter and the goblet of fire (harry potter  #4)', np.float64(0.9836461110877329)), ('harry potter and the prisoner of azkaban (harry potter  #3)', np.float64(0.9813028958725254)), ('harry potter and the order of the phoenix (harry potter  #5)', np.float64(0.98059463064796)), ('harry potter y la orden del fénix (harry potter  #5)', np.float64(0.9739214900662916)), ('harry potter and the chamber of secrets (harry potter  #2)', np.float64(0.9686653607093754)), ('harry potter and philosophy: if aristotle ran hogwarts', np.float64(0.9663156165157658)), ('angels flight (harry bosch  #6; harry bosch universe  #7)', np.float64(0.9498227279848747)), ('the harry bosch novels  volume 2: the last coyote

In [23]:
import numpy as np

common_test = test_ratings_data[
    (test_ratings_data['User_ID'].isin(predicted_ratings_df.index)) &
    (test_ratings_data['ISBN'].isin(predicted_ratings_df.columns))
    ]

def get_predicted_rating(row):
    return predicted_ratings_df.loc[row['User_ID'], row['ISBN']]

mse_value = np.mean((common_test['Rating'] - common_test.apply(get_predicted_rating, axis=1)) ** 2)

print("MSE for SVD-based Collaborative Filtering:", mse_value)

MSE for SVD-based Collaborative Filtering: 62.30996167273896


In [25]:
def predict_rating_knn(user_id, item_id, user_item_matrix, knn_cf_model, num_neighbors=30):
    if user_id not in user_item_matrix.index or item_id not in user_item_matrix.columns:
        return np.nan

    user_idx = user_item_matrix.index.get_loc(user_id)
    distances, indices = knn_cf_model.kneighbors(
        user_item_matrix.values[user_idx].reshape(1, -1),
        n_neighbors=num_neighbors+1
    )
    similar_user_indices = indices.flatten()[1:]
    similar_user_ratings = user_item_matrix.iloc[similar_user_indices]

    pred = similar_user_ratings[item_id].mean()
    return pred

def compute_mse_knn_cf(test_df, user_item_matrix, knn_cf_model, num_neighbors=30):
    preds = []
    actuals = []
    for row in test_df.itertuples():
        user_id = row.User_ID
        isbn = row.ISBN
        actual_rating = row.Rating

        predicted_rating = predict_rating_knn(user_id, isbn, user_item_matrix, knn_cf_model, num_neighbors)
        if not np.isnan(predicted_rating):
            preds.append(predicted_rating)
            actuals.append(actual_rating)

    preds = np.array(preds)
    actuals = np.array(actuals)
    mse_val = np.mean((actuals - preds) ** 2)
    return mse_val

test_mse_knn = compute_mse_knn_cf(test_ratings_data, train_user_item_matrix, knn_cf)
print("MSE for User-based kNN:", test_mse_knn)

MSE for User-based kNN: 62.59076435545889
